In [1]:
from shaolin_graphs import *

In [2]:
class GraphPlotBokeh(ToggleMenu):
    
    def __init__(self, gc, name='scatter_plot', **kwargs):
        self._width = 1200
        self._height = 800
        self._plot_title = "Graph plot"
        node_mapper_dict = {'size':{'max':100,
                            'min':1,
                            'step':0.5,
                            'high':75,
                            'low':25,
                            'default':12,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'line_width':{'max':50,
                            'min':0,
                            'step':0.5,
                            'high':5,
                            'low':1,
                            'default':2,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'fill_alpha':{'max':1.0,
                            'min':0.,
                            'step':0.05,
                            'high':0.95,
                            'low':0.3,
                            'default':1.,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'line_alpha':{'max':1.0,
                            'min':0.,
                            'step':0.05,
                            'high':0.95,
                            'low':0.3,
                            'default':1.,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'line_color':{'default_color':'black','map_data':False,'step':0.05,'min':0.0,'low':0.0},
                       'fill_color':{'default_color':'#11D4CA','map_data':False,'step':0.05,'min':0.0,'low':0.0}
                      }
        edge_mapper_dict = {'size':{'max':100,
                            'min':5,
                            'step':0.5,
                            'high':20,
                            'low':7,
                            'default':7,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'line_width':{'max':50,
                            'min':0,
                            'step':0.5,
                            'high':5,
                            'low':1,
                            'default':2,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'fill_alpha':{'max':1.0,
                            'min':0.,
                            'step':0.05,
                            'high':0.95,
                            'low':0.3,
                            'default':1.,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'line_alpha':{'max':1.0,
                            'min':0.,
                            'step':0.05,
                            'high':0.95,
                            'low':0.3,
                            'default':1.,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'line_color':{'default_color':'black','map_data':False,'step':0.05,'min':0.0,'low':0.0},
                       'fill_color':{'default_color':'#EDB021','map_data':False,'step':0.05,'min':0.0,'low':0.0}
                      }
        gc.name = 'gc'
        output_notebook(hide_banner=True)
        edge_mapper_data = gc.edge
        node_mapper = PlotMapper(gc.node, mapper_dict=node_mapper_dict, name='node_mapper', mode='interactive')
        edge_mapper = PlotMapper(edge_mapper_data, mapper_dict=edge_mapper_dict, name='edge_mapper', mode='interactive')
        node_tooltip_data = gc.node
        node_tooltip = BokehDataFrameTooltip(node_tooltip_data, name='node_tooltip')
        edge_tooltip = BokehDataFrameTooltip(edge_mapper_data, name='edge_tooltip')
        
        ToggleMenu.__init__(self,
                            children=[Dashboard(["r$n=buton_dash",["btn$d=Update&n=update_btn",'tog$d=Plot text']],name='plot_ctrl'),gc,node_mapper,
                                            edge_mapper,
                                            node_tooltip,
                                            edge_tooltip],
                            name=name,**kwargs)
        #self._init_layout()
        #self.observe(self.update)
        self.gc.calculate.observe(self.update)
        self.node_mapper.marker.marker_type.observe(self._on_marker_change)
        self.edge_mapper.marker.marker_type.observe(self._on_marker_change)
        self.plot_ctrl.plot_text.observe(self._on_marker_change)
        self.plot_ctrl.update_btn.observe(self._on_marker_change)
        self._handler = None
    
    def prepare_edge_mapper_data(self,gc):
        et = gc.edge.copy()
        cols = et.items.values
        fi = pd.DataFrame(et[cols[0]].unstack().copy())
        fi.columns = [cols[0]]

        for it in cols[1:]:
            new = pd.DataFrame(et[it].unstack().copy())
            new.columns = [it]
            fi = fi.combine_first(new)
        fi.index.levels[0].name = 'major'
        fi.index.levels[1].name = 'minor'
        dft = fi.reset_index()
        
        ti = list(zip(dft.minor.values,dft.major.values))
        dft.index = ti
        dft = dft.drop(['major','minor'],axis=1)
        return dft.ix[gc.G.edges()].copy()
        
    def _init_layout(self):
        params = ['size', 'line_width', 'fill_alpha', 'line_alpha', 'line_color', 'fill_color']
        for target in ['node','edge']:
            mapper = getattr(self,target+'_mapper')
            getattr(mapper).data_slicer.index_slicer.visible = False
            
    def update_source_df(self):
        """Datasources for plots managing"""
        def index_to_columns(df):
            old_index = df.index
            new_index = pd.MultiIndex.from_tuples(df.index)
            df.index = new_index
            df = df.reset_index()
            df.index = old_index
            columns = df.columns.values
            columns[:2] = ['major','minor']
            df.columns = columns
            return df
        def fix_pos_index(edge_pos,edge_source):
            changed = set(edge_source.index) - set(edge_pos.index)
            edge_source.index = pd.MultiIndex.from_tuples(edge_source.index)
            edge_pos.index = pd.MultiIndex.from_tuples(edge_pos.index)
            for e in changed:
                inv = (e[1],e[0])
                edge_pos.loc[inv,'minor'] = e[1]
                edge_pos.loc[inv,'major'] = e[0]
            return edge_pos
        self.node_source= self.node_mapper.output.join(self.node_tooltip.output)
        #self.edge_tooltip.output.index = pd.MultiIndex.from_tuples(self.edge_tooltip.output.index.values)
        #self.edge_mapper.output.index = pd.MultiIndex.from_tuples(self.edge_mapper.output.index.values)
        self.edge_source= self.edge_mapper.output.join(self.edge_tooltip.output) 
        
        node_pos = self.gc.layout.node['2d'].dropna(axis=1).copy()
        #edge_pos = self.gc.layout.edge['2d'].copy()
        
        self.output_node = pd.concat([self.node_source, node_pos], axis=1).fillna('NaN').copy()
        self.output_node.columns = [str(x) for x in self.output_node.columns]
        self.output_node.index = [str(x) for x in self.output_node.index]
        df = self.edge_source.copy()
        df['src'] = self.gc.edge['src'].values
        df['dst'] = self.gc.edge['dst'].values
        lay = self.gc.layout.edge['2d'].fillna(0).copy()
        #df.loc[:,'index'] = df['index'].map(str).values
        lay = lay.reset_index()
        lay['src'] = lay['index'].map(lambda x:x[0])
        lay['dst'] = lay['index'].map(lambda x:x[1])
        """for c in lay.columns:
            df[c] = np.nan
        rows = lay.iterrows()
        for ix,row in rows:
            cond_src = df['src']==ix[0]
            cond_dst = df['dst']==ix[1]
            df.loc[cond_src&cond_dst,lay.columns] = row.values"""
        self.output_edge = df.merge(lay,on=['src','dst'], left_index=True).fillna('NaN').copy()#edge_pos.merge(self.edge_source,on=['major','minor'],left_index=True)
        def _hack(x):
            try:
                return float(x)
            except:
                return str(x)
        self.output_edge = self.output_edge.applymap(_hack)
    
    def update_mappers(self):
        
        edge_mapper_data = self.gc.edge
        self.edge_mapper.data = edge_mapper_data
        self.node_mapper.data = self.gc.node
        self.node_tooltip.data = self.gc.node
        self.edge_tooltip.data = edge_mapper_data
    
    def update_tooltips(self):
        """Update tooltip data and inject it into the plot tooltips"""
        self.plot.tools[-1].tooltips = self.edge_tooltip.create_tooltip()
        self.plot.tools[-2].tooltips = self.node_tooltip.create_tooltip()

    def update(self, _=None):
        """Set up all the combined elements needed for the plot"""
        self.node_mapper.update()
        self.edge_mapper.update()
        self.update_mappers()
        self.update_source_df()
        self.update_tooltips()
        self.push_data()
        
    def push_data(self):
        """A function to push the content of the source DataFrame
        to a specific plot source
        """
        for col in self.output_node.columns:
            self.node_bokeh_source.data[col] = self.output_node[col].values
        
        for col in self.output_edge.columns:
            self.edge_bokeh_source.data[col] = self.output_edge[col].values
        push_notebook()
    
    def _on_marker_change(self, _=None):
        #clear_output()
        #self.init_plot()
        self.update()
        #return display(show(self.plot))
    
    def init_plot(self):
        """Handle plot init"""
        self.node_bokeh_source = bks.ColumnDataSource(self.output_node)
        self.edge_bokeh_source = bks.ColumnDataSource(self.output_edge)
        n_ttip = self.node_tooltip.create_tooltip()
        e_ttip = self.edge_tooltip.create_tooltip()

        self.plot = figure(width=self._width, height=self._height, webgl=False,
                           tools="pan,wheel_zoom,box_zoom,reset,resize,crosshair,save",)

        #node_marker = 'circle'#self.node_marker_free_sel.marker.value
        #edge_marker = self.edge_marker_free_sel.marker.value
        self.plot.title.text  = self._plot_title
        self.plot.segment('x0', 'y0',
                          'x1', 'y1',
                          color='line_color',
                          line_width='line_width',
                          alpha='line_alpha',
                          source=self.edge_bokeh_source)

        edg_center = self.plot.scatter(source=self.edge_bokeh_source,
                                       x='cx', y='cy',
                                       line_color='line_color',
                                       fill_color='fill_color',
                                       alpha='fill_alpha',
                                       size='size',
                                       line_width='line_width',
                                       marker=self.edge_mapper.marker.marker_type.value
                                       )#,
                                       #marker=edge_marker)
        
        nod = self.plot.scatter(source=self.node_bokeh_source,
                                x='x', y='y',
                                line_color='line_color',
                                fill_color='fill_color',
                                fill_alpha='fill_alpha',
                                size='size',
                                line_width='line_width',#,
                                marker=self.node_mapper.marker.marker_type.value)
    
        if self.plot_ctrl.plot_text.value:
            self.plot.text(source=self.node_bokeh_source,
                           x='x', y='y', y_offset=8,
                           text='label',
                           text_align='center')

        self.plot.add_tools(HoverTool(tooltips=n_ttip, renderers=[nod]))
        self.plot.add_tools(HoverTool(tooltips=e_ttip, renderers=[edg_center]))
        self.plot.title.align = "center"
        self.plot.title.text_font_size = "22px"      
        #no grid 
        self.plot.xgrid.grid_line_color = None
        self.plot.ygrid.grid_line_color = None
        #y axis
        self.plot.yaxis.axis_line_width = 0
        self.plot.yaxis.major_tick_line_color = None
        self.plot.yaxis.major_tick_line_width = 0
        self.plot.yaxis.minor_tick_line_color = None
        #self.plot.yaxis.axis_label_text_font_size = "18px"
        
        #self.plot.yaxis.major_label_text_font_size = "14px"
        #self.plot.yaxis[0].formatter = NumeralTickFormatter(format="0%")
       
        self.plot.yaxis.axis_line_width = 0
        self.plot.yaxis.axis_line_color = None
        self.plot.yaxis.major_label_text_font_size = "0px"
        self.plot.yaxis.axis_label_text_font_size = "0px"
        self.plot.yaxis.axis_label = ''
        #x axis
        self.plot.xaxis.axis_line_width = 0
        self.plot.xaxis.major_tick_line_color = None
        self.plot.xaxis.major_tick_line_width = 0
        self.plot.xaxis.minor_tick_line_color = None
        #self.plot.yaxis.axis_label_text_font_size = "18px"
        
        #self.plot.yaxis.major_label_text_font_size = "14px"
        #self.plot.yaxis[0].formatter = NumeralTickFormatter(format="0%")
       
        self.plot.xaxis.axis_line_width = 0
        self.plot.xaxis.axis_line_color = None
        self.plot.xaxis.major_label_text_font_size = "0px"
        self.plot.xaxis.axis_label_text_font_size = "0px"
        self.plot.xaxis.axis_label = ''
        self._handler = show(self.plot, notebook_handle=True)
    @property
    def snapshot(self, name='bokeh_scatter'):
        html =  notebook_div(self.plot)
        widget = shaoscript('html$N='+name)
        widget.value = html
        return widget

    def load_data(self):
        self.node_mapper.update()
        self.edge_mapper.update()
        self.update_mappers()
        
        self.update_source_df()
        self.init_plot()
        """ self.node_mapper.observe(self.update)
        self.edge_mapper.observe(self.update)
        self.node_tooltip.observe(self.update)
        self.edge_tooltip.observe(self.update)
        self.gc.calculate.observe(self.update)"""
        #self.update_tooltips()
       
    
    def show(self):
        return display(self.widget, show(self.plot))

In [3]:
%%HTML
<style>
.container{width:90% !important;}
</style>

In [4]:
import pickle

with open('geant_graph.pck', 'rb') as handle:
    g = pickle.load(handle)


In [9]:
gm = GraphCalculator(g,metrics=True)

In [10]:
gp = GraphPlotBokeh(gm)
gp._plot_title = "Geant's betweenness centrality"

In [25]:
#gp._plot_title = "Geant's eigenvector centrality"
gp.load_data()
#gp.init_plot()
gp.widget

In [28]:
snap = gp.snapshot.value

import pickle
with open('geant_data/geant_betweenness_new.pck', 'wb') as handle:
    pickle.dump(snap, handle)

In [29]:
from IPython.core.display import HTML
HTML(snap)

In [40]:
gp.plot.text?